<a href="https://colab.research.google.com/github/antoniomarquina/tensorflow/blob/master/Red_Neuronal_Profunda_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Red Neuronal "Profunda" en PyTorch**"

Recordar cambiar el entorno de ejecución a GPU o TPU si trabajamos en Colab

### Descarga de las dependencias

In [ ]:
import torch
import torch.nn as nn

from torchvision.datasets import MNIST
from torchvision import transforms

from torchsummary import summary

#### Descarga de datos

In [ ]:
train = MNIST('data', train=True, transform=transforms.ToTensor(), download=True)
test = MNIST('data', train=False, transform=transforms.ToTensor())

#### Batch data

In [ ]:
train_loader = torch.utils.data.DataLoader(train, batch_size=128) 
test_loader = torch.utils.data.DataLoader(test, batch_size=128) 

#### Diseño de la arquitectura de la red neuronal

In [ ]:
n_input = 784
n_dense_1 = 64
n_dense_2 = 64
n_dense_3 = 64
n_out = 10

In [ ]:
model = nn.Sequential(
    
    # first hidden layer: 
    nn.Linear(n_input, n_dense_1), 
    nn.ReLU(), 
    
    # second hidden layer: 
    nn.Linear(n_dense_1, n_dense_2), 
    nn.ReLU(), 
    
    # third hidden layer: 
    nn.Linear(n_dense_2, n_dense_3), 
    nn.ReLU(), 
    nn.Dropout(),  
    
    # output layer: 
    nn.Linear(n_dense_3, n_out) 
)

In [ ]:
summary(model, (1, n_input))

#### Configuración de los hiper-parámetros de entrenamiento

In [ ]:
cost_fxn = nn.CrossEntropyLoss() # includes softmax activation

In [ ]:
optimizer = torch.optim.Adam(model.parameters())

#### Entrenamiento

In [ ]:
def accuracy_pct(pred_y, true_y):
  _, prediction = torch.max(pred_y, 1) # returns maximum values, indices; fed tensor, dim to reduce
  correct = (prediction == true_y).sum().item()
  return (correct / true_y.shape[0]) * 100.0

In [ ]:
n_batches = len(train_loader)
n_batches

In [ ]:
n_epochs = 10 

print('Training for {} epochs. \n'.format(n_epochs))

for epoch in range(n_epochs):
  
  avg_cost = 0.0
  avg_accuracy = 0.0
  
  for i, (X, y) in enumerate(train_loader): # enumerate() provides count of iterations  
    
    # forward propagation:
    X_flat = X.view(X.shape[0], -1)
    y_hat = model(X_flat)
    cost = cost_fxn(y_hat, y)
    avg_cost += cost / n_batches
    
    # backprop and optimization via gradient descent: 
    optimizer.zero_grad() # set gradients to zero; .backward() accumulates them in buffers
    cost.backward()
    optimizer.step()
    
    # calculate accuracy metric:
    accuracy = accuracy_pct(y_hat, y)
    avg_accuracy += accuracy / n_batches
    
    if (i + 1) % 100 == 0:
      print('Step {}'.format(i + 1))
    
  print('Epoch {}/{} complete: Cost: {:.3f}, Accuracy: {:.1f}% \n'
        .format(epoch + 1, n_epochs, avg_cost, avg_accuracy)) 

print('Training complete.')

#### Validación con los datos de grupo de prueba

In [ ]:
n_test_batches = len(test_loader)
n_test_batches

In [ ]:
model.eval() # disables dropout and batch norm

with torch.no_grad(): # disables autograd, reducing memory consumption
  
  avg_test_cost = 0.0
  avg_test_acc = 0.0
  
  for X, y in test_loader:
    
    # make predictions: 
    X_flat = X.view(X.shape[0], -1)
    y_hat = model(X_flat)
    
    # calculate cost: 
    cost = cost_fxn(y_hat, y)
    avg_test_cost += cost / n_test_batches
    
    # calculate accuracy:
    test_accuracy = accuracy_pct(y_hat, y)
    avg_test_acc += test_accuracy / n_test_batches

print('Test cost: {:.3f}, Test accuracy: {:.1f}%'.format(avg_test_cost, avg_test_acc))

# model.train() # 'undoes' model.eval()